In [1]:
from IPython.core.display import Markdown
from aocd.models import Puzzle

from common.inputreader import InputReader

puzzle = Puzzle(year=2024, day=int("06"))

display(Markdown(f"# {puzzle.title}"))
display(Markdown(f"[Open Website]({puzzle.url})"))

# example = get_code_block(puzzle, 5)

# Guard Gallivant

[Open Website](https://adventofcode.com/2024/day/6)

In [2]:
from common.matrix import Direction, MatrixNavigator

direction_char_map = {
    Direction.UP: "^",
    Direction.DOWN: "v",
    Direction.LEFT: "<",
    Direction.RIGHT: ">"
}


# test case (part 1)
def part_1(input: InputReader, debug: bool) -> int:
    matrix = input.matrix()

    def find_guard() -> (MatrixNavigator, Direction):
        for x, y, value in matrix:
            for direction, char in direction_char_map.items():
                if value == char:
                    return MatrixNavigator(matrix, x, y), direction
        return None, None

    pointer, direction = find_guard()
    positions = set()

    while True:
        positions.add(pointer.current_position)

        if debug:
            # matrix.print()
            display(f"x,y={pointer.current_position}, direction={direction_char_map[direction]}")

        ok, value = pointer.peek_value(direction)
        if not ok:
            # leaving the matrix
            break

        if value == "#":
            # turn right
            if direction == Direction.UP:
                direction = Direction.RIGHT
            elif direction == Direction.RIGHT:
                direction = Direction.DOWN
            elif direction == Direction.DOWN:
                direction = Direction.LEFT
            elif direction == Direction.LEFT:
                direction = Direction.UP
        else:
            pointer.set_value(".")
            pointer.move(direction)
            pointer.set_value(direction_char_map[direction])

    return len(positions)


example = puzzle.examples[0]
result = part_1(InputReader(example.input_data), False)
assert result == 41

In [3]:
# real case (part 1)
result = part_1(InputReader(puzzle.input_data), False)
display(result)
assert result == 5409

5409

In [8]:
# test case (part 2)
def part_2(input: InputReader, debug: bool) -> int:
    matrix = input.matrix()

    def find_guard() -> (MatrixNavigator, Direction):
        for next_x, next_y, value in matrix:
            for direction, char in direction_char_map.items():
                if value == char:
                    return MatrixNavigator(matrix, next_x, next_y), direction
        return None, None

    start_pointer, start_direction = find_guard()

    def run_simulation(pointer, direction) -> (bool, set):
        positions = set()
        history = set()
        limit = 10000

        while limit > 0:
            limit -= 1
            positions.add(pointer.current_position)
            x, y = pointer.current_position
            vector = (x, y, direction)
            if vector in history:
                return True, positions
            history.add(vector)

            if debug:
                # matrix.print()
                display(f"x,y={pointer.current_position}, direction={direction_char_map[direction]}")

            ok, value = pointer.peek_value(direction)
            if not ok:
                return False, positions

            if value == "#":
                # turn right
                if direction == Direction.UP:
                    direction = Direction.RIGHT
                elif direction == Direction.RIGHT:
                    direction = Direction.DOWN
                elif direction == Direction.DOWN:
                    direction = Direction.LEFT
                elif direction == Direction.LEFT:
                    direction = Direction.UP
            else:
                pointer.set_value(".")
                pointer.move(direction)
                pointer.set_value(direction_char_map[direction])

        # throw error if the limit is reached
        raise Exception("Limit reached")

    # run initial simulation
    start_x, start_y = start_pointer.current_position
    _, locations = run_simulation(MatrixNavigator(matrix, start_x, start_y), start_direction)
    looping_positions = 0

    # loop over every position except the starting one
    for x, y in list(locations):
        # copy matrix
        new_matrix = matrix.copy()

        # modify the new matrix putting a wall at the next position
        new_matrix.set_value(x, y, "#")

        # run simulation
        try:
            is_loop, new_positions = run_simulation(MatrixNavigator(new_matrix, start_x, start_y), start_direction)
        except Exception as e:
            raise e

        # if it's a loop add that to the looping positions
        if is_loop:
            looping_positions += 1

    return looping_positions


example = puzzle.examples[0]
result = part_2(InputReader(example.input_data), False)
assert result == 6

In [9]:
# real case (part 2)
result = part_2(InputReader(puzzle.input_data), False)
display(result)
assert result == 2022

2022